In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

import json
from tqdm import tqdm_notebook as tqdm

In [2]:
url_search = 'https://rating.chgk.info/api/tournaments.json/search?name={}'
url_tournament = 'https://rating.chgk.info/api/tournaments/{}/list'
url_recaps = 'https://rating.chgk.info/api/tournaments/{idtournament}/recaps/{idteam}'
url_player = 'https://rating.chgk.info/api/players/{}'

Change the search query here

In [3]:
#search = 'чемпионат эстонии'
search = 'кубок европы'

In [4]:
response = requests.get(url_search.format(search)).text
response = json.load(StringIO(response))
tournaments = pd.DataFrame(response['items'])

In [5]:
def get_player_stat_tournament(idtournament):
    tournament_results = json.load(StringIO(requests.get(url_tournament.format(idtournament)).text))
    player_results = []
    
    for team_result in tournament_results:
        position = float(team_result['position'])
        team_recaps = json.load(StringIO(requests.get(url_recaps.format(idtournament=idtournament, idteam=team_result['idteam'])).text))
        # Collect player stats
        for player in team_recaps:
            player_result = {'idplayer': player['idplayer'],
                             'idteam': team_result['idteam'],
                             'team_name': team_result['base_name'],
                             'position': position,
                             'questions': team_result['questions_total']}
            player_results.append(player_result)

    return pd.DataFrame(player_results)

In [6]:
tournaments.date_end[0].split('-')[0]

'2005'

In [7]:
player_results = []
for i, r in tqdm(tournaments.iterrows(), total=len(tournaments)):
    res = get_player_stat_tournament(r.idtournament)
    res['year'] = r.date_end.split('-')[0]
    player_results.append(res)

In [9]:
all_players = pd.concat(player_results)
all_players.head()

,idplayer,idteam,position,questions,team_name,year
0,8077,10,6.5,0,Яровит,2005
1,21026,10,6.5,0,Яровит,2005
2,23442,10,6.5,0,Яровит,2005
3,25516,10,6.5,0,Яровит,2005
4,28419,10,6.5,0,Яровит,2005


Ever played this tournament

In [10]:
len(all_players.idplayer.unique())

1377

Played max times

In [21]:
def get_name(idplayer):
    response = json.load(StringIO(requests.get(url_player.format(idplayer)).text))[0]
    return ' '.join([response['surname'], response['name'], response['patronymic']])

def get_names(players):
    return [get_name(i) for i in tqdm(players)]

In [12]:
tournament_counts = all_players.groupby('idplayer').count().iloc[:,0].sort_values(ascending=False)
max_times_players = tournament_counts[tournament_counts == tournament_counts.max()]
max_times_players

idplayer
32004    8
Name: idteam, dtype: int64

In [13]:
get_names(max_times_players.index.tolist())

['Топчий Иван Александрович']

In [ ]:
n_times_players_names = get_names(tournament_counts.index.tolist()[:30])
n_times_players = pd.DataFrame(tournament_counts)
n_times_players.loc[:30, 'name'] = n_times_players_names

In [25]:
n_times_players.head(30)

,idteam,name
idplayer,,
32004,8,Топчий Иван Александрович
34909,7,Чернуха Ирина Викторовна
25794,7,Порицкий Владислав Валерьевич
78734,7,Зайков Иван Анатольевич
34936,7,Чернышёв Кирилл Александрович
21560,7,Морозов Александр Владимирович
71365,6,Тух Вадим Евгеньевич
77785,6,Адаскевич Станислав Владимирович
18194,6,Кленницкая Мария Владимировна


Count medals

In [14]:
medalists = all_players.loc[all_players.position < 4,['idplayer', 'position']].reset_index(drop=True)
medalists.position = np.array(medalists.position, dtype=np.uint8)
len(medalists)

310

In [15]:
medalists.head()

,idplayer,position
0,2454,1
1,3159,1
2,4270,1
3,5130,1
4,21633,1


In [16]:
medalists_aggregated = pd.get_dummies(medalists, columns=['position']).groupby('idplayer').sum().sort_values(['position_{}'.format(i) for i in range(1,4)], ascending=False)
medalists_aggregated['name'] = get_names(medalists_aggregated.index.tolist())
medalists_aggregated

,position_1,position_2,position_3,name
idplayer,,,,
74382,3,1,0,Новосёлов Михаил Андреевич
24006,3,0,0,Панайотти Дмитрий Алексеевич
24850,2,2,0,Печеный Александр Павлович
19541,2,1,1,Малкин Михаил Леонидович
36592,2,1,0,Штых Алексей Константинович
18194,2,0,1,Кленницкая Мария Владимировна
71365,2,0,1,Тух Вадим Евгеньевич
2958,2,0,0,Белов Вячеслав Евгеньевич
33271,2,0,0,Филиппов Юрий Станиславович


In [18]:
medalists_aggregated['position_s'] = medalists_aggregated.loc[:,'position_1':'position_3'].sum(axis=1)
medalists_aggregated.sort_values(['position_s']+['position_{}'.format(i) for i in range(1,4)], ascending=False)

,position_1,position_2,position_3,name,position_s
idplayer,,,,,
932,1,2,2,Алхазов Григорий Эдуардович,5
74382,3,1,0,Новосёлов Михаил Андреевич,4
24850,2,2,0,Печеный Александр Павлович,4
19541,2,1,1,Малкин Михаил Леонидович,4
70750,1,3,0,Галиакберов Денис Андреевич,4
72160,1,3,0,Карчевский Александр Сергеевич,4
34909,0,3,1,Чернуха Ирина Викторовна,4
20998,0,2,2,Миронов Денис Эдуардович,4
24006,3,0,0,Панайотти Дмитрий Алексеевич,3


In [26]:
medalists_aggregated.to_csv('./medalists_kest.csv')